In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
zip_path= '/content/drive/My Drive/ISPP_Folder/CIFAKE_Shadow_Datasets.zip'

In [3]:
import zipfile
import os

# Define the destination directory for unzipped files
extract_to = '/content/unzipped_folder/'

# Ensure the destination directory exists
os.makedirs(extract_to, exist_ok=True)

# Open and extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Files unzipped successfully.")

Files unzipped successfully.


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import random
from torch.utils.data import DataLoader, Subset, random_split
from torchvision import datasets
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL image to PyTorch tensor
])


In [5]:
train_dataset1 = datasets.ImageFolder(root="/content/unzipped_folder/CIFAKE_Shadow_Datasets/New_Train_Shadow/Dataset1", transform=transform)
train_dataset2 = datasets.ImageFolder(root="/content/unzipped_folder/CIFAKE_Shadow_Datasets/New_Train_Shadow/Dataset2", transform=transform)
train_dataset3 = datasets.ImageFolder(root="/content/unzipped_folder/CIFAKE_Shadow_Datasets/New_Train_Shadow/Dataset3", transform=transform)
train_dataset4 = datasets.ImageFolder(root="/content/unzipped_folder/CIFAKE_Shadow_Datasets/New_Train_Shadow/Dataset4", transform=transform)
train_dataset5 = datasets.ImageFolder(root="/content/unzipped_folder/CIFAKE_Shadow_Datasets/New_Train_Shadow/Dataset5", transform=transform)

In [6]:
train_loader1 = DataLoader(train_dataset1, batch_size = 1, shuffle = True)
train_loader2 = DataLoader(train_dataset2, batch_size = 1, shuffle = True)
train_loader3 = DataLoader(train_dataset3, batch_size = 1, shuffle = True)
train_loader4 = DataLoader(train_dataset4, batch_size = 1, shuffle = True)
train_loader5 = DataLoader(train_dataset5, batch_size = 1, shuffle = True)

In [7]:
test_dataset1 = datasets.ImageFolder(root="/content/unzipped_folder/CIFAKE_Shadow_Datasets/New_Test_Shadow/Dataset1",transform=transform)
test_dataset2 = datasets.ImageFolder(root="/content/unzipped_folder/CIFAKE_Shadow_Datasets/New_Test_Shadow/Dataset2",transform=transform)
test_dataset3 = datasets.ImageFolder(root="/content/unzipped_folder/CIFAKE_Shadow_Datasets/New_Test_Shadow/Dataset3",transform=transform)
test_dataset4 = datasets.ImageFolder(root="/content/unzipped_folder/CIFAKE_Shadow_Datasets/New_Test_Shadow/Dataset4",transform=transform)
test_dataset5 = datasets.ImageFolder(root="/content/unzipped_folder/CIFAKE_Shadow_Datasets/New_Test_Shadow/Dataset5",transform=transform)

In [9]:
test_loader1 = DataLoader(test_dataset1, batch_size=1, shuffle=True)
test_loader2 = DataLoader(test_dataset2, batch_size=1 ,shuffle=True)
test_loader3 = DataLoader(test_dataset3, batch_size=1, shuffle=True)
test_loader4 = DataLoader(test_dataset4, batch_size=1, shuffle=True)
test_loader5 = DataLoader(test_dataset5, batch_size=1, shuffle=True)

In [20]:
test_loaders = [test_loader1, test_loader2, test_loader3, test_loader4, test_loader5]
train_loaders = [train_loader1, train_loader2, train_loader3, train_loader4, train_loader5]

In [10]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.pool(self.tanh(self.conv1(x)))
        x = self.pool(self.tanh(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)  # Flatten the tensor for the fully connected layer
        x = self.tanh(self.fc1(x))
        x = self.fc2(x)
        return x

In [11]:
shadow_model1 = CNN()
shadow_model2 = CNN()
shadow_model3 = CNN()
shadow_model4 = CNN()
shadow_model5 = CNN()

In [13]:
shadow_model1.load_state_dict(torch.load('shadow_model_1.pt', map_location=torch.device('cpu')))
shadow_model2.load_state_dict(torch.load('shadow_model_2.pt', map_location=torch.device('cpu')))
shadow_model3.load_state_dict(torch.load('shadow_model_3.pt', map_location=torch.device('cpu')))
shadow_model4.load_state_dict(torch.load('shadow_model_4.pt', map_location=torch.device('cpu')))
shadow_model5.load_state_dict(torch.load('shadow_model_5.pt', map_location=torch.device('cpu')))

<ipython-input-13-1c8551d50487>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  shadow_model1.load_state_dict(torch.load('shadow_model_1.pt', map_location=torch.device('cpu

<All keys matched successfully>

In [19]:
shadow_models = [shadow_model1,shadow_model2, shadow_model3, shadow_model4, shadow_model5]

In [14]:
softmax = nn.Softmax(dim=1)

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cpu


In [18]:
def generate_attack_data(model, data_loader, member_label):
    model.eval()
    attack_data = []
    attack_labels = []
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            probs = softmax(model(images))
            attack_data.append(probs.cpu().numpy())
            attack_labels.append(member_label)
    return attack_data, attack_labels

In [22]:
attack_data = []
attack_labels = []
for i in range(5):
  member_data, member_labels = generate_attack_data(shadow_models[i], train_loaders[i], member_label=1)
  non_member_data, non_member_labels = generate_attack_data(shadow_models[i], test_loaders[i], member_label=0)
  attack_data.extend(member_data+non_member_data)
  attack_labels.extend(member_labels+non_member_labels)

In [23]:
import numpy as np

In [24]:
attack_data = np.array(attack_data).reshape(len(attack_data), -1)
attack_labels = np.array(attack_labels)

In [25]:
np.savez('attack_model_train_data.npz', X=attack_data, Y=attack_labels)

In [26]:
print(attack_data.shape)
print(attack_labels.shape)

(60000, 10)
(60000,)
